### 14장 모델 성능 향상시키기

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 와인데이터
df = pd.read_csv('./data/wine.csv', header=None)
X = df.iloc[:,0:12]
y = df.iloc[:,12]

# 학습셋, 테스트셋 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# 모델 구조 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델이 저장되는 조건 설정
modelpath = "./data/model/all/{epoch:02d}-{val_accuracy:.4f}.keras"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

# 모델 실행
history = model.fit(X_train, y_train, epochs=2000, batch_size=500, validation_split=0.25, 
                    verbose=1, callbacks=[early_stopping_callback, checkpointer]) 

# 테스트 결과 출력
score = model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

hist_df = pd.DataFrame(history.history)
hist_df

# val_loss, loss값들 저장
y_vloss = hist_df['val_loss']
y_loss = hist_df['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, "o", c='red', markersize=2, label='Testset_loss')
plt.plot(x_len, y_loss, "o", c='blue', markersize=2, label='Trainset_loss')

plt.legend(loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


### 15장 실제 데이터로 만들어 보는모델

1)주택가격 데이터

In [ ]:
df = pd.read_csv("./data/house_train.csv")
df

# 각 데이터가 어떤 유형으로 되어 있는지? - 정수,실수,오브젝트형 등등 다양함
df.dtypes

# 판다스의 get_dummies()를 이용해서 카테고리형 변수를 0, 1로 이루어진 변수로 change (원핫 인코딩)
df = pd.get_dummies(df)

# 결측치 대체
df = df.fillna(df.mean())

# 데이터 사이의 상관관계 저장
df_corr = df.corr()

# *집값과 관련이 큰 것부터 순서대로 저장
df_corr_sort = df_corr.sort_values('SalePrice', ascending=False)

# 집값을 제외한 나머지 열을 저장한다
cols_train = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF']
X_train_pre = df[cols_train]

# 집값을 저장
y = df['SalePrice'].values

# 학습:테스트=8:2
X_train, X_test, y_train, y_test = train_test_split(X_train_pre, y, test_size=0.2)

# 모델 구조 설정
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(1))
model.summary()

# 모델을 실행
model.compile(optimizer='adam', loss='mean_squared_error')

# 20번 이상 결과가 향상되지 않으면 자동으로 중단
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델의 이름 설정
modelpath = './data/model/Ch15-house.keras'

# 최적화된 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                               verbose=0, save_best_only=True)

# 실행 관련 설정을 하는 부분 - 전체의 20%를 검증셋으로 설정
history = model.fit(X_train, y_train, validation_split=0.25, epochs=2000,
                    batch_size=32, callbacks=[early_stopping_callback, checkpointer])


In [ ]:
# 주택 가격 예측 모델
## 학습 결과를 시각화하기 위해 예측 값과 실제 값, 실행 번호가 들어갈 빈 리스트를 만들고,
## 25개의 샘플로부터 얻은 결과를 채워 넣는다
real_prices = []
pred_prices = [] # 예상가
X_num = []

n_iter = 0
Y_prediction = model.predict(X_test).flatten()

for i in range(25):
    real = y_test[i]
    prediction = Y_prediction[i]
    print('실제가격: {:.2f}, 예상가격: {:.2f}'.format(real, prediction))
    real_prices.append(real)

    pred_prices.append(prediction)
    n_iter += 1
    X_num.append(n_iter)

# 이제 위 결과를 그래프를 통해 비교해보자
plt.plot(X_num, pred_prices, label='predicted price')
plt.plot(X_num, real_prices, label='real price')
plt.legend()
plt.show()

2)추가데이터 - car

In [ ]:
train_df = pd.read_excel('https://github.com/cranberryai/todak_todak_python/blob/master/machine_learning/regression/carprice_E1SUl6b.xlsx?raw=true', sheet_name='train')
test_df = pd.read_excel('https://github.com/cranberryai/todak_todak_python/blob/master/machine_learning/regression/carprice_E1SUl6b.xlsx?raw=true', sheet_name='test')

### 16장 이미지 인식의 꽃, 컨볼루션신경망(CNN)

1)mnist데이터

In [35]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import os
import matplotlib.pyplot as plt
import sys

In [ ]:
# MNIST 데이터셋을 불러와 학습&테스트셋으로 저장
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 학습셋과 테스트셋이 각각 몇 개의 이미지로 되어 있는지 화깅ㄴ
print('학습셋 이미지 수: %d개' % (X_train.shape[0]))
print('테스트 이미지 수: %d개' % (X_test.shape[0]))

# 첫 번째 이미지를 확인
plt.imshow(X_train[0], cmap='Greys')
plt.show()

# 이미지가 인식되는 원리
for x in X_train[0]:
    for i in x:
        sys.stdout.write('%-3s' % i)
    sys.stdout.write('\n')

# 차원 변환 과정을 연습해보자
X_train = X_train.reshape(X_train.shape[0], 784)
X_train = X_train.astype('float32')
X_train = X_train/255
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32') / 255

# 클래스값을 확인
print('class: %d' % (y_train[0]))

# 바이너리화 과정을 연습해보자
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(y_train[0])

In [ ]:
# 딥러닝 모델 기본 프레임
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax')) # 이건 10개 output - 0~9까지 [0,0,0,0,0,1,0,0,0,0]
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # 최적화함수는 adam

# 모델 최적화를 위한 설정 구간
modelpath = './data/MNIST_MLP.keras'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델 실행
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200,
                    verbose=0, callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력
print('\n Test Accuracy: %.4f' % (model.evaluate(X_test, y_test)[1]))

# 검증셋과 학습셋의 오차를 저장 - 그래프로 보여줄거임
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

1-2)최종 코드

In [ ]:
# 16장 최종 코드

# 데이터 load & 전처리
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# *컨볼루션 신경망 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) # 10개 output

# 모델 실행(compile) 옵션 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 최적화를 위한 설정 구간
modelpath = './data/MNIST_CNN.keras'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델 실행ㄱㄱ
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200,
                    verbose=0, callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력
print('\n Test Accuracy: %.4f' % (model.evaluate(X_test, y_test)[1]))

# 검증셋과 학습셋의 오차를 저장 - 그래프로 보여줄거임
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

2)추가 데이터 - 7분 30초걸림 후.. (4번째 줄 빼고, 동일함)

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

# fashion_mnist데이터셋을 불러와 학습&테스트셋으로 저장
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# *컨볼루션 신경망 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) 

# 모델 실행(compile) 옵션 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 최적화를 위한 설정 구간
modelpath = './data/fashion_MNIST_CNN.keras'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델 실행
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200,
                    verbose=0, callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력
print('\n Test Accuracy: %.4f' % (model.evaluate(X_test, y_test)[1]))

# 검증셋과 학습셋의 오차를 저장
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

### 19장 - 세상에 없는 얼굴 GAN, 오토인코더

In [47]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 1)생성자 - 모델이름을 generator로 정한다
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2))) # relu에서 손실되는 값 조금이라도 더 보완한 함수
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

In [ ]:
# 2)판별자 - 모델이름을 discriminator로 정한다
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid')) # sigmoid를 왜 쓰냐면, 0과 1을 판별하기 때문이지!
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

discriminator.trainable = False # 학습할 것 아니니까!

In [ ]:
def gan_train(epoch, batch_size, saving_interval): # 세 가지 변수 지정
    # MNIST 데이터load. 단, 테스트는 필요없고 이미지만 사용
    (X_train, _), (_, _) = mnist.load_data()

    # 가로28픽셀, 세로28픽셀이고 흑백이므로 1을 설정
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

    # 0~255사이 픽셀값에서 127.5를 뺀 후, 127.5로 나누면 -1~1사이 값으로 바뀐다
    X_train = (X_train-127.5)/127.5

    true = np.ones((batch_size, 1)) # 1
    idx = np.random.randint(0, X_train.shape[0], batch_size) # 2
    imgs = X_train[idx] # 3
    d_loss_real = discriminator.train_on_batch(imgs, true) # 4

    fake = np.zeros((batch_size, 1)) # 1
    noise = np.random.normal(0, 1, (batch_size, 100)) # 2
    gen_imgs = generator.predict(noise) # 3
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake) # 4

    # d_loss_real, d_loss_fake의 평균이 바로 판.별.자의 오차
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import matplotlib.pyplot as plt

# 예제 파일에서 data 폴더 아래에 이미지가 저장될 gan_images 폴더가 함께 제공됩니다.
# 만약 이미지가 저장될 폴더가 없다면 아래 코드의 주석을 해제해 gan_images 폴더를 만듭니다.
# import os
# if not os.path.exists("./data/gan_images"):
#    os.makedirs("./data/gan_images")

# 생성자 모델을 만듭니다.
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

# 판별자 모델을 만듭니다.
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

# 생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

# 신경망을 실행시키는 함수를 만듭니다.
def gan_train(epoch, batch_size, saving_interval):

  # MNIST 데이터를 불러옵니다.
  (X_train, _), (_, _) = mnist.load_data()
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
  X_train = (X_train - 127.5) / 127.5  # 픽셀 값은 0에서 255 사이의 값 - 이전에 255로 나누어 줄때는 이를 0~1 사이의 값으로 바꾸었던 것인데, 여기서는 127.5를 빼준 뒤 127.5로 나누어 줌으로 인해 -1에서 1사이의 값으로 바뀌게 된다

  true = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for i in range(epoch):
    # 실제 데이터를 판별자에 입력하는 부분
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    d_loss_real = discriminator.train_on_batch(imgs, true)

    # 가상 이미지를 판별자에 입력하는 부분
    noise = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

    # 판별자와 생성자의 오차를 계산
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    g_loss = gan.train_on_batch(noise, true)

    print("여기!!!", d_loss)

    print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

    # 이 부분은 중간 과정을 이미지로 저장해 주는 부분이다. 본 장의 주요 내용과 관련이 없어 소스 코드만 첨부
    # 만들어진 이미지들은 gan_images 폴더에 저장된다
    if i % saving_interval == 0:
        #r, c = 5, 5
        noise = np.random.normal(0, 1, (25, 100))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(5, 5)
        count = 0
        for j in range(5):
            for k in range(5):
                axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                axs[j, k].axis('off')
                count += 1
        fig.savefig("./data/gan_images/gan_mnist_%d.png" % i)


gan_train(2001, 32, 200) # 2000번 반복(+1)되고, 배치 크기는 32, 200번마다 결과가 저장